In [16]:
import pandas as pd
from datetime import datetime
from dateutil import relativedelta
import datetime as dt

In [26]:
# БЛОК С ИМПОРТОМ ДАННЫХ
data=pd.read_csv("./Data/casc-resto.csv",sep=';', decimal=",") # Считываем данные из файла
data = pd.DataFrame(data)
data['RKDate']=pd.to_datetime(data['RKDate'])  # столбец с датой сделаем в формате даты
fix_date=(datetime.strptime('2017-07-01', '%Y-%m-%d'))
print(data.info())
data['SummAfterPointsUsage'] = data['SummAfterPointsUsage'].astype(float) # приведем эти столбцы к типу с плавающей запятой
data['SummBasic'] = data['SummBasic'].astype(float)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882222 entries, 0 to 882221
Data columns (total 9 columns):
CustomerID              882222 non-null int64
Restaurant              882222 non-null int64
RKDate                  882222 non-null datetime64[ns]
RegionName              882222 non-null object
BrandsNames             882222 non-null object
DishCategoryName        881608 non-null object
Quantity                882222 non-null int64
SummBasic               882222 non-null float64
SummAfterPointsUsage    882222 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 60.6+ MB
None


In [27]:
# БЛОК С РАСЧЕТОМ ПОКАЗАТЕЛЕЙ
def agg(group): # функция, в которой считаются все показатели
    bought=0 # переменная для покупал/не покупал
    recency=10000
    spent = 0
    visitedTimes = 1
    nonCashSpent = 0
    visitedTimes1 = 1
    visitsBeforeFix=0
    firstVisitDate=fix_date
    MonetaryValue=0
    Mean_quantity=0
    QuantityForGroup=sum( group['Quantity'] )
    for index, row in group.iterrows(): # пробегаемся по датам в каждой группе 
        if row['RKDate'] <=  datetime.strptime('2017-12-31', '%Y-%m-%d')  and row['RKDate'] >= fix_date: # сравниваем даты
            bought=1 # покупка была совершена в промежуток времени 
            
        if row['RKDate'] <= fix_date and (fix_date - row['RKDate']).days < recency: 
            recency = (fix_date - row['RKDate']).days # считаем количество дней от даты покупки/визита до фиксированной даты
                
        if row['RKDate'] <= fix_date:
            visitsBeforeFix=visitsBeforeFix+1 # считаем дату самого недавнего визита до фиксированной даты
            spent=spent+row['SummAfterPointsUsage'] # считаем сколько всего было потрачено
        if row['RKDate']<= firstVisitDate:
            firstVisitDate=row['RKDate'] # считаем дату первого визита 
            
        #if (row['RKDate'] <= fix_date):           
            

    if visitsBeforeFix == 0: # если не было визитов до фиксированной даты, то частота равна 0
        frequency=0
    months  = 1 + (relativedelta.relativedelta(fix_date,firstVisitDate)).months # считаем сколько месяцев прошло от первого визита до фиксированной даты
    if months!=0:
        frequency= visitsBeforeFix/ months # считаем частоту
    if visitsBeforeFix != 0:
        MonetaryValue= spent / visitsBeforeFix # считаем средний чек покупки за время до фиксированной даты  
        Mean_quantity=QuantityForGroup/visitsBeforeFix
     # среднее количество купленного товара до фиксированной даты, собственный показатель
    
    return bought,recency,frequency,MonetaryValue,Mean_quantity 

    

def func(group): #функция, в которой все посчитанные показатели собираются в один массив
    bought, recency,frequency,MonetaryValue, Mean_quantity=agg(group)
    d = {}
    d['Bought_Or_Not'] = bought
    d['Recency'] = recency
    d['Frequency'] = frequency
    d['Monetary_Value'] = MonetaryValue
    d['Mean_Quantity'] = Mean_quantity
   
    return pd.Series(d, index=['Bought_Or_Not', 'Recency','Frequency','Monetary_Value','Mean_Quantity'])

groupedData = data.groupby("CustomerID").apply(func) # группируем записи по клиенту и применяем функцию, в которой все считается
aggregatedData= groupedData.reset_index()
print(aggregatedData)



      CustomerID  Bought_Or_Not  Recency  Frequency  Monetary_Value  \
0        2728046            0.0    160.0   8.142857      251.789474   
1        2728088            0.0    720.0   4.857143      279.647059   
2        2728089            1.0     15.0   4.714286      398.734848   
3        2728095            0.0    177.0  14.142857      262.737374   
4        2728107            1.0    115.0   3.000000      393.714286   
...          ...            ...      ...        ...             ...   
9995     2913103            0.0    401.0   9.750000      443.538462   
9996     2913106            0.0    126.0  46.750000      252.983957   
9997     2913114            0.0     95.0  27.750000      311.873874   
9998     2913123            1.0      8.0  58.000000      129.232759   
9999     2913132            1.0     44.0   2.571429      370.222222   

      Mean_Quantity  
0          1.035088  
1          1.000000  
2          1.666667  
3          1.141414  
4          1.809524  
...            

In [28]:
# БЛОК СО СЛИЯНИЕМ ДАННЫХ
dataToMergeWith =pd.read_csv("./Data/CASC_constant.csv",sep=',') # Считываем данные из файла для слияния
dataToMergeWith['ActivationDate'] = pd.to_datetime(dataToMergeWith['ActivationDate'])
MergedData=pd.merge(aggregatedData, dataToMergeWith, left_on='CustomerID', right_on='CustomerId',how='inner') # Делаем слияние двух таблиц
del MergedData['CustomerId']
del MergedData['CustomerID']
Sex=pd.get_dummies(MergedData['Sex'], drop_first=True) # создадим новый столбец, в котором используем фиктивные переменные: мужской пол =1 ,женский пол = 0

MergedData.drop(['Sex'],axis=1,inplace=True) # Удаляем столбец с полом, потому что там прописано м и ж, а нам нужно 1 или 0

MergedData=pd.concat([MergedData,Sex],axis=1) # присоединяем новый столбец с полом в нужном формате к нашей таблице

MergedData['ActivationDate'] = pd.to_datetime(MergedData['ActivationDate']) # переведем формат из строки в дату

MergedData['ActivationDate']=MergedData['ActivationDate'].map(dt.datetime.toordinal) # сделаем дату числом
MergedData['Age'] = MergedData['Age'].fillna(MergedData['Age'].mean())
print(MergedData)

      Bought_Or_Not  Recency  Frequency  Monetary_Value  Mean_Quantity  \
0               0.0    160.0   8.142857      251.789474       1.035088   
1               0.0    720.0   4.857143      279.647059       1.000000   
2               1.0     15.0   4.714286      398.734848       1.666667   
3               0.0    177.0  14.142857      262.737374       1.141414   
4               1.0    115.0   3.000000      393.714286       1.809524   
...             ...      ...        ...             ...            ...   
9995            0.0    401.0   9.750000      443.538462       1.871795   
9996            0.0    126.0  46.750000      252.983957       1.224599   
9997            0.0     95.0  27.750000      311.873874       1.225225   
9998            1.0      8.0  58.000000      129.232759       1.422414   
9999            1.0     44.0   2.571429      370.222222       1.777778   

      ActivationDate   Age  SubscribedEmail  SubscribedPush  Male  
0             735599  24.0            False

In [29]:
# БЛОК С ОБУЧЕНИЕМ МОДЕЛИ
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

X_data=MergedData.drop('Bought_Or_Not',axis=1) # исключаем из данных зависимую переменную

x_train, x_test, y_train, y_test = train_test_split(X_data, MergedData['Bought_Or_Not'], test_size=0.2, random_state=42) # разделим данные, тестовая выборка составит 20%
logreg=LogisticRegression() # создадим логистическую регрессию
logreg.fit(x_train, y_train) # применим функцию fit на обучающей выборке
y_pred = logreg.predict(x_test) # применим функцию predict для предсказания вероятности прихода покупателя
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))  # посчитаем показатели precision и recall  
score = logreg.score(x_test, y_test)
print(score)

[[954 125]
 [591 330]]
              precision    recall  f1-score   support

         0.0       0.62      0.88      0.73      1079
         1.0       0.73      0.36      0.48       921

    accuracy                           0.64      2000
   macro avg       0.67      0.62      0.60      2000
weighted avg       0.67      0.64      0.61      2000

0.642


C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
